# Задача по тема 15 (Flask)

Напишете Flask API, който моделира работата със задачи.

Една **задача** съдържа следната информация:
- име (name)
- идентификационен номер (цяло число) (task-id)


Вашето Flask API трябва да има следните пътища:
- `/` - Връща отговор със стойност 200 и тяло "Hello !"
- `/add` - Добавя нова задача към системата. 
    - Очаква се, че потребителя ще подаде `name` и `task-id` в header-ите на заявката. 
    - Ако някой от тези стойности не присъства в header-ите на заявката, върнете отговор с код 400 и тяло: `Not all required headers are present.`
    - При грешка свързана с работата със запазването на информацията (повече инфо виж долу), върнете отговор с код 500 и тяло `Cannot create task`
    - При успешно добавяне на задача, върнете отговор с код 200 и тяло `Task added successfully`.
- `/remove` - Премахва задача от системата
    - Очаква се, че потребителя ще подаде `task-id` в header-ите на заявката.
    - Ако тази стойност не присъства в header-ите на заявката, върнете отговор с код 400 и тяло: `Not all required headers are present.`
    - При грешка свързана с работата със запазването на информацията (повече инфо виж долу), върнете отговор с код 500 и тяло `Cannot remove task`
    - Ако търсената задача не съществува, върнете отговор с код 200 и тяло `No task found with id=` и поисканото id
    - При успешно премахване на задача, върнете отговор с код 200 и тяло `Task removed successfully`.
- `/get` - Връща задача от системата
    - Очаква се, че потребителя ще подаде `task-id` в header-ите на заявката.
    - Ако тази стойност не присъства в header-ите на заявката, върнете отговор с код 400 и тяло: `Not all required headers are present.`
    - При грешка свързана с работата със запазването на информацията (повече инфо виж долу), върнете отговор с код 500 и тяло `Cannot read task`
    - Ако търсената задача не съществува, върнете отговор с код 200 и тяло `No task found with id=` и поисканото id
    - Ако търсената задача съществува, върнете отговор с код 200 и тяло `{name}`, където `{name}` е името на задачата
- `/reset`
    - Зачиства storage-а на системата
    - Връща отговор с код 200 и тяло `Reset storage succesfully`


Други изисквания, на които трябва вашето API да отговаря:
- Persistent storage - задачите се пазят на външна локация (файл) и не се изгубват при спиране на сървъра.
- Файлът за storage трябва да е в JSON формат
- Файлът за storage трябва да се казва `storage.json`
- Файлът за storage трябва да се намира в същата директория, в която е тази тетрадка
- Може да приемете, че няма да се пускат паралелни заявки - т.е. в даден момент от време може да бъде изпълнявана най-много една заявка
- Може да приемете, че няма да има две задачи с еднакви task-id-та
- Преди да изпълните тестовете, стартирайте Flask сървъра ръчно
- Ако адресът на сървъра ви се различава от `http://127.0.0.1:5000`, променете стойноста на променливата в тестовете

In [79]:
import os

import requests

API_URL = 'http://127.0.0.1:5000'
ADD_URL = API_URL + '/add'
REMOVE_URL = API_URL + '/remove'
GET_URL = API_URL + '/get'
RESET_URL = API_URL + '/reset'

def test_01_verify_main_page():
    # Arrange
    expected_code = 200
    expected_text = 'Hello !'

    # Act
    response = requests.get(API_URL)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 1: ✅ 0.25 points")

def test_02_add_task():
    # Arrange
    expected_code = 200
    expected_text = 'Task added successfully'

    headers = {'name': 'test_02', 'task-id': '2'}
    requests.get(RESET_URL)

    # Act
    response = requests.get(ADD_URL, headers=headers)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 2: ✅ 0.25 points")

def test_03_add_task_no_task_id():
    # Arrange
    expected_code = 400
    expected_text = 'Not all required headers are present'

    headers = {'name': 'test_02'}
    requests.get(RESET_URL)

    # Act
    response = requests.get(ADD_URL, headers=headers)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 3: ✅ 0.25 points")

def test_04_add_task_no_name():
    # Arrange
    expected_code = 400
    expected_text = 'Not all required headers are present'

    headers = {'task-id': '5'}
    requests.get(RESET_URL)

    # Act
    response = requests.get(ADD_URL, headers=headers)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 4: ✅ 0.25 points")

def test_05_add_task_invalid_storage():
    # Arrange
    expected_code = 500
    expected_text = 'Cannot create task'

    headers = {'name': 'foo', 'task-id': '2'}
    requests.get(RESET_URL)
    os.remove('storage.json')

    # Act
    response = requests.get(ADD_URL, headers=headers)
    requests.get(RESET_URL)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 6: ✅ 0.25 points")

def test_06_remove_task():
    # Arrange
    expected_code = 200
    expected_text = 'Task removed successfully'

    headers = {'task-id': '2'}
    requests.get(RESET_URL)
    requests.get(ADD_URL, headers={'name': 'foo', 'task-id': '2'})

    # Act
    response = requests.get(REMOVE_URL, headers=headers)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 6: ✅ 0.25 points")

def test_07_remove_task_no_task_id():
    # Arrange
    expected_code = 400
    expected_text = 'Not all required headers are present'

    headers = {'asd': '2'}
    requests.get(RESET_URL)
    requests.get(ADD_URL, headers={'name': 'foo', 'task-id': '2'})

    # Act
    response = requests.get(REMOVE_URL, headers=headers)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 7: ✅ 0.25 points")

def test_08_remove_task_non_existing_task():
    # Arrange
    expected_code = 200
    expected_text = 'No task found with id=3'

    headers = {'task-id': '3'}
    requests.get(RESET_URL)
    requests.get(ADD_URL, headers={'name': 'foo', 'task-id': '2'})

    # Act
    response = requests.get(REMOVE_URL, headers=headers)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 8: ✅ 0.25 points")

def test_09_remove_task_invalid_storage():
    # Arrange
    expected_code = 500
    expected_text = 'Cannot remove task'

    headers = {'task-id': '2'}
    requests.get(RESET_URL)
    os.remove('storage.json')

    # Act
    response = requests.get(REMOVE_URL, headers=headers)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 9: ✅ 0.25 points")

def test_10_get_task():
    # Arrange
    expected_code = 200
    expected_text = 'foo2'

    headers = {'task-id': '3'}
    requests.get(RESET_URL)
    requests.get(ADD_URL, headers={'name': 'foo', 'task-id': '2'})
    requests.get(ADD_URL, headers={'name': 'foo2', 'task-id': '3'})
    requests.get(ADD_URL, headers={'name': 'foo3', 'task-id': '4'})

    # Act
    response = requests.get(GET_URL, headers=headers)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 10: ✅ 0.25 points")

def test_11_get_task_no_task_id():
    # Arrange
    expected_code = 400
    expected_text = 'Not all required headers are present'

    headers = {'asd': '3'}
    requests.get(RESET_URL)
    requests.get(ADD_URL, headers={'name': 'foo', 'task-id': '2'})
    requests.get(ADD_URL, headers={'name': 'foo2', 'task-id': '3'})
    requests.get(ADD_URL, headers={'name': 'foo3', 'task-id': '4'})

    # Act
    response = requests.get(GET_URL, headers=headers)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 11: ✅ 0.25 points")

def test_12_get_task_non_existing_task():
    # Arrange
    expected_code = 200
    expected_text = 'No task found with id=42'

    headers = {'task-id': '42'}
    requests.get(RESET_URL)
    requests.get(ADD_URL, headers={'name': 'foo', 'task-id': '2'})
    requests.get(ADD_URL, headers={'name': 'foo2', 'task-id': '3'})
    requests.get(ADD_URL, headers={'name': 'foo3', 'task-id': '4'})

    # Act
    response = requests.get(GET_URL, headers=headers)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 12: ✅ 0.25 points")

def test_13_get_task_invalid_storage():
    # Arrange
    expected_code = 500
    expected_text = 'Cannot read task'

    headers = {'task-id': '4'}
    requests.get(RESET_URL)
    os.remove('storage.json')

    # Act
    response = requests.get(GET_URL, headers=headers)

    actual_code = response.status_code
    actual_text = response.text

    # Assert
    assert expected_code == actual_code
    assert expected_text == actual_text
    print("Test 13: ✅ 0.25 points")


test_01_verify_main_page()
test_02_add_task()
test_03_add_task_no_task_id()
test_04_add_task_no_name()
test_05_add_task_invalid_storage()
test_06_remove_task()
test_07_remove_task_no_task_id()
test_08_remove_task_non_existing_task()
test_09_remove_task_invalid_storage()
test_10_get_task()
test_11_get_task_no_task_id()
test_12_get_task_non_existing_task()
test_13_get_task_invalid_storage()


Test 1: ✅ 0.25 points
Test 2: ✅ 0.25 points
Test 3: ✅ 0.25 points
Test 4: ✅ 0.25 points
Test 6: ✅ 0.25 points
Test 6: ✅ 0.25 points
Test 7: ✅ 0.25 points
Test 8: ✅ 0.25 points
Test 9: ✅ 0.25 points
Test 10: ✅ 0.25 points
Test 11: ✅ 0.25 points
Test 12: ✅ 0.25 points
Test 13: ✅ 0.25 points
